## 直线检测算法汇总

在一些任务场景中，通常会用到直线检测算法，比如车道线检测、长度测量等。


### Hough_line 直线检测

Hough变换常用于直线检测、圆检测、椭圆检测等。实现步骤如下：

- 创建二维数组或累加器，并将其初始值设置为零
- 用$r$表示行，用$\theta$表示列
- 数组的大小取决于所需要的精度。假设希望角度的精度为1度，则需要180列
- 对于$r$，可能的最大距离是图像的对角线长度。因此取一个像素精度，行数可以是图像的对角线长度

In [ ]:
# coding=utf-8
# 导入相应的python包
import cv2 
import numpy as np 
import os
  
# 读取输入图片
img = cv2.imread('../image/lineDetection/line_detection.png') 
# 将彩色图片灰度化
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
# 使用Canny边缘检测 
edges = cv2.Canny(gray,50,200,apertureSize = 3) 
# 进行Hough_line直线检测
lines = cv2.HoughLines(edges,1,np.pi/10, 300) 
print(lines)
# 遍历每一个r和theta
for i in range(len(lines)):
    r,theta = lines[i, 0, 0], lines[i, 0, 1]
    # 存储cos(theta)的值
    a = np.cos(theta)
    # 存储sin(theta)的值
    b = np.sin(theta) 
    # 存储rcos(theta)的值
    x0 = a*r 
    # 存储rsin(theta)的值 
    y0 = b*r  
    # 存储(rcos(theta)-1000sin(theta))的值
    x1 = int(x0 + 1000*(-b)) 
    # 存储(rsin(theta)+1000cos(theta))的值
    y1 = int(y0 + 1000*(a)) 
    # 存储(rcos(theta)+1000sin(theta))的值
    x2 = int(x0 - 1000*(-b)) 
    # 存储(rsin(theta)-1000cos(theta))的值
    y2 = int(y0 - 1000*(a))  
    # 绘制直线结果  
    cv2.line(img,(x1,y1), (x2,y2), (0,255,0),2) 
# 保存结果
# cv2.imwrite('test3_r.jpg', img) 
cv2.imshow("result", img)
cv2.waitKey(0)


### HoughLinesP

In [10]:
# coding=utf-8
# 导入相应的python包
import cv2
import numpy as np

# 读取输入图片
img = cv2.imread('../image/lineDetection/line_detection.png') 
# 将彩色图片灰度化
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# 使用Canny边缘检测
edges = cv2.Canny(gray,50,200,apertureSize = 3)
# 进行Hough_line直线检测
lines = cv2.HoughLinesP(edges,1,np.pi/180, 80, 30, 10)

# 遍历每一条直线
for i in range(len(lines)):
	cv2.line(img,(lines[i, 0, 0],lines[i, 0, 1]), (lines[i, 0, 2],lines[i, 0, 3]), (0,255,0),2)
# 保存结果
cv2.imwrite('test3_r.jpg', img)
cv2.imshow("result", img)
cv2.waitKey(0)

-1

### LSD算法

LSD是opencv中集成的一个直线检测算法，该算法的直线检测效果优于Hough算法，而且具有较好的检测速度，推荐使用。LSD快速直线检测算法是由Rafael Grompone、Jeremie Jackbowicz、Jean-Michel Morel于2010年发表在PAMI上的文献《LSD:a Line Segment Dectctor》中提出的，该算法时间复杂度较霍夫变换低。LSD算法通过对图像局部分析，得出直线的像素点集，再通过假设参数进行验证求解，将像素点集合与误差控制集合合并，进而自适应控制误检的数量 。一般来说，要检测图像中的直线，最基本的思想是检测图像中梯度变化较大的像素点集，LSD算法也正是利用梯度信息和行列线（level-line）来进行直线检测的。

In [11]:
# coding=utf-8
import cv2
import numpy as np

# 读取输入图片
img0 = cv2.imread('../image/lineDetection/line_detection.png') 
# 将彩色图片转换为灰度图片
img = cv2.cvtColor(img0,cv2.COLOR_BGR2GRAY)

# 创建一个LSD对象
lsd = cv2.createLineSegmentDetector(0)
# 执行检测结果
dlines = lsd.detect(img)
# 绘制检测结果
for dline in dlines[0]:
    x0 = int(round(dline[0][0]))
    y0 = int(round(dline[0][1]))
    x1 = int(round(dline[0][2]))
    y1 = int(round(dline[0][3]))
    cv2.line(img0, (x0, y0), (x1,y1), (0,255,0), 1, cv2.LINE_AA)

# 显示并保存结果
cv2.imwrite('test3_r.jpg', img0)
cv2.imshow("LSD", img0)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-oduouqig\opencv\modules\imgproc\src\lsd.cpp:143: error: (-213:The function/feature is not implemented) Implementation has been removed due original code license issues in function 'cv::LineSegmentDetectorImpl::LineSegmentDetectorImpl'
